In [ ]:
# =============================================================================
# Carico i file
# =============================================================================
train_file = r'data/training.txt'
train_url=r'data/training1.csv'
test_file = r'data/test.txt'
test_url = r'data/test1.csv'
delimiter = ','



In [ ]:
import csv
x_train = list()
y_train = list()
topic_train = list()
with open(train_url, encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile, delimiter=delimiter)
    for row in reader:
        x_train.append(row[5])
        y_train.append(row[2])
        topic_train.append(row[3])

x_test = list()
y_test = list()
topic_test = list()
with open(test_url, encoding='utf-8', newline='') as infile:
    reader = csv.reader(infile, delimiter=delimiter)
    for row in reader:
        x_test.append(row[5])
        y_test.append(row[2])
        topic_test.append(row[3])
        
len(x_train),len(y_train),len(x_test),len(y_test)
y_train.pop(0) # elimino il primo elemento che è la parola "gender"
y_test.pop(0) # elimino il primo elemento che è la parola "gender"
set(y_train)
sample_idx = 10

x_train.pop(0) # elimino il primo elemento che è la parola "post"
x_test.pop(0) # elimino il primo elemento che è la parola "post"
x_train[sample_idx]
y_train[sample_idx]

topic_train.pop(0) # elimino il primo elemento che è "topic"
topic_test.pop(0) # elimino il primo elemento che è "topic"

In [ ]:
# =============================================================================
# Seleziono il topic            commentare questa parte per prendere tutti topic
# =============================================================================


topics = df["topic"].unique()
print(topics)
from collections import Counter
Counter(df["topic"])
# Seleziono il training set
indices = [i for i, x in enumerate(topic_train) if x == "SPORTS"] 
x_train = [x_train[i] for i in indices]
y_train = [y_train[i] for i in indices]
# len(x_train), len(y_train)
print("Lunghezza del training set: ", len(x_train))
print("Percentuale di 0-19 nel training set: ", round(y_train.count("0-19") / len(y_train), 3))
print("Percentuale di 20-29 nel training set: ", round(y_train.count("20-29") / len(y_train), 3))
print("Percentuale di 30-39 nel training set: ", round(y_train.count("30-39") / len(y_train), 3))
print("Percentuale di 40-49 nel training set: ", round(y_train.count("40-49") / len(y_train), 3))
print("Percentuale di 50-100 nel training set: ", round(y_train.count("50-100") / len(y_train), 3))

# Seleziono il test set
indices = [i for i, x in enumerate(topic_test) if x == "SPORTS"]
x_test = [x_test[i] for i in indices]
y_test = [y_test[i] for i in indices]
# len(x_test), len(y_test)
print("Lunghezza del test set: ", len(x_test))
print("Percentuale di 0-19 nel test set: ", round(y_test.count("0-19") / len(y_test), 3))
print("Percentuale di 20-29 nel test set: ", round(y_test.count("20-29") / len(y_test), 3))
print("Percentuale di 30-39 nel test set: ", round(y_test.count("30-39") / len(y_test), 3))
print("Percentuale di 40-49 nel test set: ", round(y_test.count("40-49") / len(y_test), 3))
print("Percentuale di 50-100 nel test set: ", round(y_test.count("50-100") / len(y_test), 3))

In [ ]:
# =============================================================================
# Operazioni sul testo
# =============================================================================
# Rendo minuscola la prima parola dopo il punto
import string
import re

# Elimino la parola "post"
regex = re.compile("\\bpost\\b")
for i in range(0, len(x_train)):
    x_train[i] = regex.sub('', x_train[i])
for i in range(0, len(x_test)):
    x_test[i] = regex.sub('', x_test[i])
# Elimino la punteggiatura
regex = re.compile('[%s]' % re.escape(string.punctuation))
for i in range(0, len(x_train)):
    x_train[i] = regex.sub('', x_train[i])
for i in range(0, len(x_test)):
    x_test[i] = regex.sub('', x_test[i])
# Elimino i numeri
regex = re.compile("[0-9]+")
for i in range(0, len(x_train)):
    x_train[i] = regex.sub('', x_train[i])
for i in range(0, len(x_test)):
    x_test[i] = regex.sub('', x_test[i])

In [ ]:
# =============================================================================
# Funzioni
# =============================================================================
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

stopword_list = stopwords.words('italian')

from collections import defaultdict
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV
lemmatizer = WordNetLemmatizer()

doc_counter = 0
def reset_counter():
    global doc_counter
    doc_counter = 0

def increase_counter():
    global doc_counter
    doc_counter += 1
    if doc_counter % 100 == 0:
        print(doc_counter)

def nltk_ngram_tokenizer(text):
    increase_counter()

    # tokens, skipping stopwords
    tokens = [token for token in word_tokenize(text) if token not in stopword_list]

    # per creare gli ngrams
    bigrams = ['BI_'+w1+'_'+w2 for w1,w2 in nltk.ngrams(tokens,2)]
    trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(tokens,3)]

    all_tokens = list()
    all_tokens.extend(tokens)
    all_tokens.extend(bigrams)
    all_tokens.extend(trigrams)
    return all_tokens

def nltk_nlp_tokenizer(text):
    increase_counter()

    # tokens, skipping stopwords
    tokens = [token for token in word_tokenize(text) if token not in stopword_list]

    # lemmatized tokens
    lemmas = list()
    for token, tag in pos_tag(tokens):
  	    lemmas.append('LEMMA_'+lemmatizer.lemmatize(token, tag_map[tag[0]]))

    # nltk function per creare gli ngrams
    lemma_bigrams = ['BI_'+p1+'_'+p2 for p1,p2 in nltk.ngrams(lemmas,2)]
    lemma_trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(lemmas,3)]

    all_tokens = list()
    all_tokens.extend(lemmas)
    all_tokens.extend(lemma_bigrams)
    all_tokens.extend(lemma_trigrams)
    return all_tokens


In [ ]:
# Rendo la risposta binaria
import numpy as np
# y_train[sample_idx] è una delle due classi, nel nostro caso M. Quando vado a fare y_train == y_train[sample_idx], in pratica
# metto TRUE per i maschi (M) e FALSE per le femmine (F).
y_train_bin = np.asarray(y_train)==y_train[sample_idx]
y_test_bin = np.asarray(y_test)==y_train[sample_idx]
y_train_bin,y_test_bin

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
# vect = CountVectorizer(analyzer=nltk_ngram_tokenizer, min_df=5)  # tokenization and frequency count
# vect = CountVectorizer(analyzer=nltk_n_gram_tokenizer)  
# vect = CountVectorizer(analyzer=nltk_nlp_tokenizer, min_df=5)  
vect = CountVectorizer(analyzer = nltk_nlp_tokenizer, min_df=5) # Passiamo la funzione spacy_nlp_tokenizer per ottenere sia parole singole che n-grams
reset_counter()
X_train_tok = vect.fit_transform(x_train)
reset_counter()
X_test_tok = vect.transform(x_test)

len(vect.vocabulary_)
X_train_tok[:5]
print(X_train_tok[:5])
vect.inverse_transform(X_train_tok[:5])
for feat,freq in zip(vect.inverse_transform(X_train_tok[:5])[1],X_train_tok[:5].data):
  print(feat,freq)

# FEATURE SELECTION
bin_sel = SelectKBest(chi2, k=150)
bin_sel.fit(X_train_tok,y_train_bin)
X_train_sel_bin = bin_sel.transform(X_train_tok)
X_test_sel_bin = bin_sel.transform(X_test_tok)

bin_sel.get_support()
X_train_sel_bin
print(X_train_sel_bin[:5])
print(vect.inverse_transform(bin_sel.inverse_transform(X_train_sel_bin[:5])))

# PESI CON TF-IDF
tfidf = TfidfTransformer()
tfidf.fit(X_train_sel_bin)
X_train_vec_bin = tfidf.transform(X_train_sel_bin)
X_test_vec_bin =tfidf.transform(X_test_sel_bin)

print(X_train_vec_bin[:5])
for feat,weight,freq in zip(vect.inverse_transform(bin_sel.inverse_transform(X_train_vec_bin[:5]))[1],X_train_vec_bin[:5].data,X_train_sel_bin[:5].data):
  print(feat,weight,freq)

In [ ]:
# =============================================================================
# Linear SVC
# =============================================================================
pipeline = Pipeline([
    ('sel', SelectKBest(chi2, k=150)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

classifier = pipeline.fit(X_train_tok,y_train)
predictions = classifier.predict(X_test_tok)
correct = 0
for prediction,true_label in zip(predictions, y_test):
    if prediction==true_label:
        correct += 1
print(correct/len(predictions))

from sklearn.metrics import confusion_matrix, classification_report
print('Classification report per il LinearSVC:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

In [ ]:
# =============================================================================
# One Vs One
# =============================================================================
from sklearn.multiclass import OneVsOneClassifier

pipeline = Pipeline([
    ('sel', SelectKBest(chi2, k=150)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', OneVsOneClassifier(LinearSVC()))  # learning algorithm
])

classifier = pipeline.fit(X_train_tok,y_train)
predictions = classifier.predict(X_test_tok)

from sklearn.metrics import confusion_matrix, classification_report
print('Classification report del OneVsOne con il LinearSVC:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

In [ ]:
# =============================================================================
# Logistic Regression 
# =============================================================================
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('sel', SelectKBest(chi2, k=150)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LogisticRegression(solver="newton-cg",multi_class="multinomial", class_weight="balanced"))  # learning algorithm
])

classifier = pipeline.fit(X_train_tok,y_train)
predictions = classifier.predict(X_test_tok)

from sklearn.metrics import confusion_matrix, classification_report
print('Classification report della Logistic Regression:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
   

    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X, y = load_digits(return_X_y=True)

title = "Learning Curves (OneVsOne(LinearSVC))"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = OneVsOneClassifier(LinearSVC())
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

title = r"Learning Curves (LogisticRegression)"

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = LogisticRegression(solver="saga",multi_class="multinomial", class_weight="balanced")
plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

plt.show()